In [1]:
import pandas as pd
from pyspark.sql import *

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

jugadores = spark.read.csv("players.csv",header=True).rdd
jugadores = jugadores.map(lambda x:(pd.to_numeric(x[0],"coerce"),x[1],pd.to_datetime(x[2],"coerce"),x[3])).cache()

estadisticas = spark.read.csv("stats.csv",header=True).rdd
estadisticas = estadisticas.map(lambda x:(pd.to_numeric(x[0],"coerce"),pd.to_numeric(x[1],"coerce"),\
                             pd.to_numeric(x[2],"coerce"),pd.to_numeric(x[3],"coerce"),\
                             pd.to_numeric(x[4],"coerce"))).cache()

partidos = spark.read.csv("matches.csv",header=True).rdd
partidos = partidos.map(lambda x:(pd.to_numeric(x[0],"coerce"),pd.to_numeric(x[1],"coerce"),\
                       pd.to_numeric(x[2],"coerce"),pd.to_numeric(x[3],"coerce"),x[4],x[5])).cache()

In [3]:
#jugadores: (ID_JUGADOR,Nombre_apellido,fecha_nacimiento,país)
#estadisticas: (ID_PARTIDO,ID_JUGADOR,tiempo_juego,distancia_recorrida, goles)
#partidos: (ID_PARTIDO,año,semestre,nro_fecha,equipo_local, equipo_visitante)

In [ ]:
def generar_registros(x):
    if x[0][1]==1:
        return [x,((x[0][0]-1,2,x[0][2]),x[1])]
    return [x,((x[0][0],1,x[0][2]),x[1])]
    
jugadores_argentina = jugadores.filter(lambda x:(lambda x: x[3]=="Argentina")).\
        map(lambda x:(x[0],x[1]))#(id,nombre y apellido)
semestre_partidos = partidos.map(lambda x:(x[0],(x[1],x[2])))#(id partido,(año,semestre))
jugador_anio_semestre = estadisticas.map(lambda x:(x[0],(x[1],x[2]))).\
        join(semestre_partidos)#(id partido,((id jugador,tiempo de juego),(año,semestre)))
tiempo_jugador_semestre = jugador_anio_semestre.\
        map(lambda x:((x[1][1][0],x[1][1][1],x[1][0][0]),x[1][0][1]))#((año,semestre,id jugador),tiempo de juego)
tiempo_total_semestre = tiempo_jugador_semestre.\
        reduceByKey(lambda x,y:x+y)#((año,semestre,id jugador),tiempo de juego)
tiempo_total_semestre.flatMap(generar_registros).reduceByKey()

mal mal mal mal mal mal mal mal mal mal mal